In [ ]:
import cv2
import numpy as np
from numpy.core.numeric import True_
from numpy.lib.arraysetops import unique
import pandas as pd
import glob
import os
import json
import datetime
import matplotlib.pyplot as plt


make_video = False
dataset_dir_path = './datasets/au-air/'
save_path = './tmp/'
annotations_path = dataset_dir_path + 'auair2019annotations/'
image_path = dataset_dir_path + 'auair2019data/images/'
annotations_file = annotations_path + 'annotations.json'
video_max_frames = 2000

video_ext = '.avi'
image_ext = '.jpg'
image_files = image_path + '*' + image_ext


In [ ]:
# def AddDateTime(names):
#     s = pd.Series(names.str.split('_'))
#     df_tmp = pd.DataFrame(s.tolist(), columns=['head', 'time', 'x', 'id.jpg'])
#     s = pd.Series(df_tmp['id.jpg'].str.split('.'))
#     df_tmp2 = pd.DataFrame(s.tolist(), columns=['id', 'jpg'])
#     df_full['time'] = df_tmp['time']
#     df_full['id'] = df_tmp2['id']
#     df_full.set_index('image_name')

## Creating annotations dataframe

In [ ]:
# using annotations:
print("Loading annotations...")
data = json.load((open(annotations_file)))
df_json = pd.json_normalize(data, 'annotations').sort_values(by=['image_name','time.ms'])
# df_bbox = pd.json_normalize(
#     data, ['annotations', 'bbox'])
# print()
# # print(data.keys())
categoriesDict = data['categories']
# print(len(df_json), len(df_bbox))
# print(df_bbox)
print(
    f'Number for annotation duplicates is {df_json.image_name.duplicated().sum()}')


## Creating Image Dataframe

In [ ]:


# df_json_date = df_json[df_json['image_name'].str.contains(search_date)]

# using images:
print("Finding image names...")
files = np.array(glob.glob(image_files))
df_imgs = pd.concat([pd.DataFrame([os.path.basename(file)], columns=['image_name']) for file in files],
                    ignore_index=True)
df_imgs['full_path'] = files
s = pd.Series(df_imgs['image_name'].str.split('_'))
df_tmp = pd.DataFrame(s.tolist(), columns=['head', 'short_name', 'x', 'id.jpg'])
s = pd.Series(df_tmp['id.jpg'].str.split('.'))
df_tmp2 = pd.DataFrame(s.tolist(), columns=['id', 'jpg'])
df_imgs['short_name'] = df_tmp['short_name']
df_imgs['id'] = df_tmp2['id']
df_imgs.set_index('image_name')
print(
    f'Number for image duplicates is {df_imgs.image_name.duplicated().sum()}')

# df_imgs = df_imgs.sort_values(by=['time', 'id']).drop_duplicates(
#     subset=['time', 'id'])
# year = df_json['time.year']
# month = df_json['time.month']
# day = df_json['time.day']
# min = df_json['time.min']
# sec = df_json['time.sec']
# msec = df_json['time.ms']
# print(year.values)
# df_json['datetime'] = pd.Timestamp(
#     str(year.values) + '-' + str(month.values) + '-' + str(day.values))
# # 'time.year', 'time.month', 'time.day', 'time.hour', 'time.min', 'time.sec')

# print(df_json['datetime'])


## Merging dataframes

In [ ]:
df_full = pd.merge(df_json, df_imgs, how='inner', on='image_name')
df_grp = df_full.groupby(
    by=['time.year', 'time.month', 'time.day', 'time.hour', 'time.min', 'time.sec'])
print("Date grouping: /n")
print(df_grp.size())


plt.hist(df_full['short_name'], bins=8)
plt.xticks(rotation = 90) 
plt.show()

## Finding number of recordings based on date

In [ ]:
unique_dates = unique(df_full['short_name'])
recording_cnt = len(unique_dates)
print(f"There are {recording_cnt} video recordings:")
print(unique_dates)

## Create a video (if needed)

In [ ]:
from video_utils import VideoUtils

vu = VideoUtils()

# Creates the video
make_video = False


# do for loop here on date
cnt = recording_cnt
for i in range(0, cnt):
    search_date = unique_dates[i]
    print('####### START #########')
    print(f'Video # {i + 1} / {cnt}')
    print(f'Selecting date: {search_date}')
    df_recording = df_full[df_full['image_name'].str.contains(search_date)]
    images = pd.Series(df_recording['full_path'].values,
                       index=df_recording['full_path'])

# Calculate Frames Per Second
    timespan_ms = df_recording['time.ms'].iloc[-1] - \
        df_recording['time.ms'].iloc[0]
    timespan_sec = timespan_ms/1000
    print(
        f'Recording duration is {timespan_sec} sec')
    fps = len(df_recording)/timespan_sec
    print(f'fps: {fps}')

    if(make_video):
        video_name = 'video_' + search_date
        if os.path.exists(video_name + video_ext):
            os.remove(video_name + video_ext)
        vu.CreateVideo(images, fps, video_name, video_ext, video_max_frames)

    print('######## END ##########')


In [ ]:
# from video_utils import VideoUtils
# vu = VideoUtils() 
# save_dir =  './new_video_20190829091111_2/'     
# created_images_array = vu.SplitVideoToFrames(save_dir, 'video_20190829091111_2.avi')

# Construct annotated video step by step

## 1.  Video selection

In [ ]:
# from video_utils import VideoUtils
# import pandas as pd
# import numpy as np

# vu = VideoUtils(categoriesDict) 

# rec_timestamp = '20190829091111' #this should be the same as unique_dates[0]
# recording_name_postfix = 2 #this should be the same as unique_dates[0]
# video_ext = '.avi'
# save_dir =  './new_video_' + rec_timestamp + '_' + str(recording_name_postfix) + '/' 
# orig_video_name =  'video_' + rec_timestamp + '_' +  str(recording_name_postfix) + video_ext
# new_video_name =  'annotated_' + rec_timestamp + '_' +  str(recording_name_postfix) + video_ext


## 2.  Split video to frames

In [ ]:
# created_images_array, cnt = vu.SplitVideoToFrames(save_dir, orig_video_name)
# images = pd.Series(created_images_array)

## 3.  Find relevent annotations

In [ ]:
# ##############
# # create image array from reading the actual images
# #img_array = vu.CreateImagesArray(images, len(created_images_array))
# print(rec_timestamp)
# # select the annotations that are only relevent to the current recording
# frames_cnt = len(created_images_array)
# df_recording_bbox = df_full[df_full['short_name'] == rec_timestamp][['time.ms','bbox']]
# # remove all non relevent data
# start_index = max((recording_name_postfix-1)*video_max_frames, 0)
# end_index = start_index + frames_cnt
# # print(start_index)
# # print(df_recording_bbox)
# df_bbox = df_recording_bbox.iloc[start_index:end_index]

# # print(df_bbox)

## 4. Add annotations to frames

In [ ]:
# for i_img in range(len(images)):
#   frame_name = images[i_img]
#   bbox_data_for_img = df_bbox['bbox'].iloc[i_img]
#   vu.AddAllFrameAnnotations(frame_name, bbox_data_for_img, 2)

## 5.  Create video

In [ ]:
# # Calculate Frames Per Second
# timespan_ms = df_bbox['time.ms'].iloc[-1] - df_bbox['time.ms'].iloc[0]
# timespan_sec = timespan_ms/1000
# fps = len(df_bbox)/timespan_sec

# if os.path.exists(new_video_name + video_ext):
#     os.remove(new_video_name + video_ext)

# vu.CreateVideo(images, fps, new_video_name, video_max_frames)

## 6. Cleanup, remove frames

In [ ]:
# #clears all frames in folder
# files_in_directory = os.listdir(save_dir)
# filtered_files = [file for file in files_in_directory if file.endswith(".jpg")]
# for file in filtered_files:
# 	path_to_file = os.path.join(save_dir, file)
# 	os.remove(path_to_file)

In [ ]:
from video_utils import VideoUtils
from video_utils import DirectoryUtils
import pandas as pd
import numpy as np

vUtils = VideoUtils(categoriesDict) 
drUtils = DirectoryUtils()

rec_timestamp = '20190905112522' #this should be the same as unique_dates[0]
recording_name_postfix = 2 #this should be the same as unique_dates[0]
video_ext = '.avi'
save_dir =  './new_video_' + rec_timestamp + '_' + str(recording_name_postfix) + '/' 
orig_video_name =  'video_' + rec_timestamp + '_' +  str(recording_name_postfix) + video_ext
new_video_name =  'annotated_' + rec_timestamp + '_' +  str(recording_name_postfix) + video_ext
video_max_frames = 2000

start_index = max((recording_name_postfix-1)*video_max_frames, 0)
end_index = start_index + video_max_frames
df_bbox = df_full[df_full['short_name'] == rec_timestamp][['bbox']].iloc[start_index:end_index]

vUtils.AnnotateVideo(save_dir, orig_video_name, new_video_name, df_bbox['bbox'].to_numpy(), 30, 2000)

drUtils.ClearFileType(save_dir,".jpg")

In [ ]:
# import datashader as ds
# import pandas as pd
# import colorcet
# # df  = pd.read_csv('census.csv')
# cvs = ds.Canvas(plot_width=850, plot_height=500)
# agg = cvs.points(df_json, 'longitude', 'latitude')
# img = ds.tf.shade(agg, cmap=colorcet.fire, how='log')